In [1]:
import dask
import dask.dataframe as ddf
import os
from glob import glob
import pandas as pd
import numpy as np
import re
import pickle

/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/home/ubuntu/.local/lib/python3.8/site-packages/dask/dataframe/utils.py:374: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


In [2]:
df = pd.read_parquet('../../../../intermediate-result-new-select-editors/editor-profile/revert-info/')

In [3]:
with open('../../all-data-article-clustering/articles-100-threshold.pkl' ,'rb') as f:
    selected_articles = pickle.load(f)

In [4]:
selected_talks = {'Talk:' + i for i in selected_articles}

In [5]:
selected_articles.update(selected_talks)

In [6]:
df = df[df['page.title'].isin(selected_articles)]

In [7]:
def page_type(title):
    if title.startswith('Talk:'):
        return 'Talk'
    return 'Article'

In [8]:
df['page.type'] = df['page.title'].apply(page_type)

In [9]:
article = df[df['page.type'] == 'Article']

In [10]:
article.columns = article.columns + '.Article'

In [11]:
article = article.rename(columns={'contributor.username.Article':'contributor.username'})

In [12]:
article = article.groupby('contributor.username').sum()

In [13]:
talk = df[df['page.type'] == 'Talk']

In [14]:
talk.columns = talk.columns + '.Talk'

In [15]:
talk = talk.rename(columns={'contributor.username.Talk':'contributor.username'})

In [16]:
talk = talk.groupby('contributor.username').sum()

In [17]:
article.index.name = ''

In [18]:
talk.index.name = ''

In [19]:
article = article.T

In [20]:
talk = talk.T

In [21]:
res = pd.concat([article,talk]).fillna(0)

In [23]:
res.to_parquet('result/revert-info-all-data')